In [2]:
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime as dt

# Parameters
rivers_reference = pd.read_csv('RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

Forecast_df = pd.DataFrame()

for i in range(len(Geodata['value']['timeSeries'])):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    print(baseStation)

    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
    except:
        KeyError
    forecast = forecast_json['properties']['periods']
    Forecast_df_temp = pd.DataFrame(forecast).drop(columns='number')
    Forecast_df_temp['Lat'] = lat
    Forecast_df_temp['Long'] = long

    Forecast_df_temp['Station'] = baseStation
    Forecast_df = Forecast_df.append(Forecast_df_temp,ignore_index=True)

Forecast_df.shape

E WALKER RV NR BRIDGEPORT, CA
E WALKER RV NR BRIDGEPORT, CA
E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
TRUCKEE R A FARAD CA
TRUCKEE R A FARAD CA
TRUCKEE R A FARAD CA
SAN LORENZO R A BIG TREES CA
SAN LORENZO R A BIG TREES CA
SAN LORENZO R A SANTA CRUZ CA
SAN LORENZO R A SANTA CRUZ CA
PIT R BL PIT NO 1 PH NR FALL RIVER MILLS CA
PIT R BL PIT NO 1 PH NR FALL RIVER MILLS CA
HAT C NR HAT CREEK CA
HAT C NR HAT CREEK CA
HAT C NR HAT CREEK CA
N YUBA R BL GOODYEARS BAR CA
N YUBA R BL GOODYEARS BAR CA
YUBA R NR MARYSVILLE CA
YUBA R NR MARYSVILLE CA
NF AMERICAN R A NORTH FORK DAM CA
NF AMERICAN R A NORTH FORK DAM CA
PUTAH C NR WINTERS CA
PUTAH C NR WINTERS CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
RUSSIAN R A HAC

(1050, 15)

In [13]:
df = Forecast_df.drop_duplicates()
df.startTime = pd.to_datetime(df.startTime)
Forecast_df.head()

/Users/scottstandring/opt/miniconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast,Lat,Long,Station
0,Tonight,2022-03-09T22:00:00-08:00,2022-03-10T06:00:00-08:00,False,13,F,None,10 to 20 mph,N,"https://api.weather.gov/icons/land/night/snow,...",Chance Light Snow,"A chance of snow. Mostly cloudy, with a low ar...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
1,Thursday,2022-03-10T06:00:00-08:00,2022-03-10T18:00:00-08:00,True,34,F,None,10 to 20 mph,N,"https://api.weather.gov/icons/land/day/snow,20...",Slight Chance Light Snow then Mostly Sunny,A slight chance of snow before 7am. Mostly sun...,38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
2,Thursday Night,2022-03-10T18:00:00-08:00,2022-03-11T06:00:00-08:00,False,11,F,None,5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 11. Northwest ...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
3,Friday,2022-03-11T06:00:00-08:00,2022-03-11T18:00:00-08:00,True,50,F,None,5 mph,SW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 50. Southwest wind aro...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
4,Friday Night,2022-03-11T18:00:00-08:00,2022-03-12T06:00:00-08:00,False,20,F,None,5 mph,SW,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 20. Southwest wind ar...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"


In [6]:
df.icon.unique()

array(['https://api.weather.gov/icons/land/night/snow,30/snow,40?size=medium',
       'https://api.weather.gov/icons/land/day/snow,20/sct?size=medium',
       'https://api.weather.gov/icons/land/night/few?size=medium',
       'https://api.weather.gov/icons/land/day/few?size=medium',
       'https://api.weather.gov/icons/land/night/skc?size=medium',
       'https://api.weather.gov/icons/land/night/sct?size=medium',
       'https://api.weather.gov/icons/land/day/sct?size=medium',
       'https://api.weather.gov/icons/land/day/bkn?size=medium',
       'https://api.weather.gov/icons/land/night/rain/snow?size=medium',
       'https://api.weather.gov/icons/land/day/snow?size=medium',
       'https://api.weather.gov/icons/land/night/snow/sct?size=medium',
       'https://api.weather.gov/icons/land/night/snow,30?size=medium',
       'https://api.weather.gov/icons/land/night/rain?size=medium',
       'https://api.weather.gov/icons/land/day/rain?size=medium',
       'https://api.weather.gov/icon